In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time as t
import pandas as pd
import os

current_directory = os.getcwd()

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

browser = webdriver.Chrome(executable_path="chromedrivers/chromedriver.exe", options=options)

In [3]:
def get_game_logs(team, year):
    browser.get(f"https://www.baseball-reference.com/teams/tgl.cgi?team={team}&t=b&year={year}")
    gamesTable = browser.find_element_by_xpath('//*[@id="team_batting_gamelogs"]')
    games = gamesTable.find_elements_by_xpath("tbody/tr")
    print("Finding game links...")
    links = []
    for g in games:
        try:
            link = g.find_element_by_xpath("td[2]").find_element_by_xpath("a").get_attribute("href")
            links.append(link)
        except selenium.common.exceptions.NoSuchElementException:
            continue
    gameCount = 1
    logs = []
    header = None
    for link in links:
        print(f"Logging Game {gameCount}...", end='\r')
        log, header = log_parser(link, browser, team)
        logs += log
        gameCount += 1
        
    output_directory = os.path.join(current_directory, "Team Batting Logs")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    df = pd.DataFrame(logs)
    writer = pd.ExcelWriter(f"Team Batting Logs/{team}.xlsx", engine='xlsxwriter')
    df.to_excel(writer, sheet_name="Game Log", header=header, index=False)
    writer.save()
        
    
def log_parser(link, browser, team):
    filename = link.split("/")[-1].split(".")[0]
    output_directory = os.path.join(current_directory, team)
    
    browser.get(link)
    table = browser.find_element_by_xpath('//table[@id="play_by_play"]')
    game = []
    header = table.find_element_by_xpath("thead/tr")
    header = [item.text for item in header.find_elements_by_xpath("th")]

    rows = table.find_elements_by_xpath("tbody/tr")
    for row in rows:
        head = row.find_element_by_xpath("th").text
        if not head or head[0] != "b" and head[0] != "t":
            continue
        items = [item.text for item in row.find_elements_by_xpath("td")]
        if items[5] != team:
            continue
        game.append([head] + items)
        
    return game, header

In [4]:
get_game_logs("SEA", 2021)

Finding game links...
